In [1]:
import pandas as pd
import datetime as datetime
import os
# ! pip install xlsxwriter --user

In [2]:
data_path=r'D:\PythonDev\MyQuantFinProject\Data'
data_file='World_Jan-Mar-24.csv'
isPy=False 
isPriceExcel=False

# Specify Period and Timefame to caculate fund performance and

In [3]:
from_str='2023-12-01'  
to_str='2024-03-31'  
timeframe='M'
#timeframe='W-FRI'

# Input data

In [4]:
if isPy:
    print(f"Data csv file in Path: {data_path}")
    data_file=input("CSV file name : ")

    from_str=input("From_date(2020-10-01) : ") 
    to_str=input("To_date(2022-12-31): ") 
    timeframe=input("Timeframe(W-FRI,M,Q,Y): ") 
  
    print(f"Process data from {from_str} to {to_str} in {timeframe} timeframe")
    press_y=input(f"Press y=True and n=False to run analaystics : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()


In [5]:
report_prefix_name=data_file.split('.')[0]
data_file=os.path.join(data_path,data_file)
if os.path.exists(data_file)==False:
    raise Exception(f"not found {data_file}")
else:
    print(f"Process file {data_file}")
print(data_file)

Process file D:\PythonDev\MyQuantFinProject\Data\World_Jan-Mar-24.csv
D:\PythonDev\MyQuantFinProject\Data\World_Jan-Mar-24.csv


# Read csv file and extract and transform data as need

In [6]:
df = pd.read_csv(data_file,index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df.index.set_names('date',inplace=True)
df.sort_index(inplace=True)
df=df.rename(columns={'Ticker':'symbol','close':'price'})
df=df.loc[from_str:to_str,['symbol','price']]
print(df.info())

list_symbol=df['symbol'].unique().tolist() 
print(list_symbol)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 296 entries, 2023-12-01 to 2024-03-19
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  296 non-null    object 
 1   price   296 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.9+ KB
None
['AAXJ', 'GLD', 'ACWI', 'SPY']


In [7]:
df.head(len(list_symbol)*3)

,symbol,price
date,,
2023-12-01,AAXJ,65.60
2023-12-01,GLD,192.01
2023-12-01,ACWI,98.75
2023-12-01,SPY,459.10
2023-12-04,GLD,187.86
2023-12-04,AAXJ,64.76
2023-12-04,SPY,456.69
2023-12-04,ACWI,98.14
2023-12-05,AAXJ,64.35


In [8]:
df.tail(len(list_symbol)*3)

,symbol,price
date,,
2024-03-15,GLD,199.71
2024-03-15,SPY,509.83
2024-03-15,AAXJ,67.49
2024-03-15,ACWI,107.82
2024-03-18,GLD,200.03
2024-03-18,SPY,512.86
2024-03-18,ACWI,108.26
2024-03-18,AAXJ,67.65
2024-03-19,ACWI,107.90


# Basic

# Resample data and calculate Percentage of change

In [9]:
colsROC=['symbol','date','pct_change']
df_Pct_Price=pd.DataFrame(columns=colsROC)
df_Pct_Price

,symbol,date,pct_change


In [10]:
for symbol_name in list_symbol:

    dfx_symbol=df.query('symbol==@symbol_name')
    print(f"===================================={symbol_name}====================================")

    dfx_px=dfx_symbol.resample(timeframe).last()
    dfx_px=dfx_px.reset_index()
    print(f"List {timeframe} Price")
    print(dfx_px)

    dfx_px['pct_change']=dfx_px['price'].pct_change()*100
    dfx_px['pct_change']=dfx_px['pct_change'].round(2)

    dfx_px=dfx_px.dropna() # the first  rown 
    dfx_px=dfx_px.drop(columns=['price']) # remove price becase we are focuse ROC of Price
    dfx_px=dfx_px[colsROC]
    
    print(f"Calculate ROC of {symbol_name}")
    print(dfx_px)
    
    df_Pct_Price=pd.concat([df_Pct_Price,dfx_px],ignore_index=True,axis=0)

====================================AAXJ====================================
List M Price
        date symbol  price
0 2023-12-31   AAXJ  66.57
1 2024-01-31   AAXJ  63.05
2 2024-02-29   AAXJ  65.86
3 2024-03-31   AAXJ  66.93
Calculate ROC of AAXJ
  symbol       date  pct_change
1   AAXJ 2024-01-31       -5.29
2   AAXJ 2024-02-29        4.46
3   AAXJ 2024-03-31        1.62
====================================GLD====================================
List M Price
        date symbol   price
0 2023-12-31    GLD  191.17
1 2024-01-31    GLD  188.45
2 2024-02-29    GLD  189.31
3 2024-03-31    GLD  199.32
Calculate ROC of GLD
  symbol       date  pct_change
1    GLD 2024-01-31       -1.42
2    GLD 2024-02-29        0.46
3    GLD 2024-03-31        5.29
====================================ACWI====================================
List M Price
        date symbol   price
0 2023-12-31   ACWI  101.77
1 2024-01-31   ACWI  102.05
2 2024-02-29   ACWI  106.65
3 2024-03-31   ACWI  107.90
Calculate ROC of 

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_16924\38245592.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dfx_px=dfx_symbol.resample(timeframe).last()
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_16924\38245592.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_Pct_Price=pd.concat([df_Pct_Price,dfx_px],ignore_index=True,axis=0)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_16924\38245592.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dfx_px=dfx_symbol.resample(timeframe).last()
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_16924\38245592.py:6: FutureWarning: 'M' is deprecated and will be removed in a future ver

In [11]:
print("==============================================================")
print("List calculate Percentage of change")
print(df_Pct_Price)

List calculate Percentage of change
   symbol       date  pct_change
0    AAXJ 2024-01-31       -5.29
1    AAXJ 2024-02-29        4.46
2    AAXJ 2024-03-31        1.62
3     GLD 2024-01-31       -1.42
4     GLD 2024-02-29        0.46
5     GLD 2024-03-31        5.29
6    ACWI 2024-01-31        0.28
7    ACWI 2024-02-29        4.51
8    ACWI 2024-03-31        1.17
9     SPY 2024-01-31        1.59
10    SPY 2024-02-29        5.22
11    SPY 2024-03-31        0.67


# Sum percentage of fund return

In [12]:
print("==============================================================")
print("Sum percentage of fund return")
dfSum_Pct_Price=df_Pct_Price.groupby(['symbol'])[['pct_change']].sum()
dfSum_Pct_Price.reset_index(drop=False,inplace=True)
dfSum_Pct_Price=dfSum_Pct_Price.sort_values(by="pct_change",ascending=False)
dfSum_Pct_Price["pct_change"]=dfSum_Pct_Price["pct_change"].round(1)
dfSum_Pct_Price

Sum percentage of fund return
  symbol  pct_change
3    SPY         7.5
1   ACWI         6.0
2    GLD         4.3
0   AAXJ         0.8


In [13]:
# writer=pd.ExcelWriter( "BasicFundPerformanceReport.xlsx",engine='openpyxl') 

# df_Pct_Price.to_csv(f"ETF-{timeframe}-Return.csv",index=False)
# dfSum_Pct_Price.to_csv(f"ETF-{timeframe}-TotalReturn.csv",index=False)


# Advance

# Pivot Percent Return To Dataframe

In [15]:
print("==============================================================")
print("Pivot Percent Return To Dataframe")
dfPct=df_Pct_Price.pivot(index='date', columns='symbol', values='pct_change')
# print(dfPct.info())
dfPct

Pivot Percent Return To Dataframe


symbol,AAXJ,ACWI,GLD,SPY
date,,,,
2024-01-31,-5.29,0.28,-1.42,1.59
2024-02-29,4.46,4.51,0.46,5.22
2024-03-31,1.62,1.17,5.29,0.67


# Take Percent Return to Find Rank

In [16]:
print("==============================================================")
print("Take Percent Return to Find Rank")
dfRank=dfPct[dfPct.columns.tolist()].rank(method='max',axis=1)
dfRank

Take Percent Return to Find Rank


symbol,AAXJ,ACWI,GLD,SPY
date,,,,
2024-01-31,1.0,3.0,2.0,4.0
2024-02-29,2.0,3.0,1.0,4.0
2024-03-31,3.0,2.0,4.0,1.0


# Combine Both return and Rank

In [18]:
print("==============================================================")
print("Combine Both return and Rank")
dfSummary=pd.concat([\
                     dfPct.sum().to_frame(name='sum_return'),\
                     dfRank.sum().to_frame(name='sum_rank'),\
                    ],axis=1)
dfSummary.fillna(value=0,inplace=True)
dfSummary

Combine Both return and Rank


,sum_return,sum_rank
symbol,,
AAXJ,0.79,6.0
ACWI,5.96,8.0
GLD,4.33,7.0
SPY,7.48,9.0


# Summarize Total Score 

In [19]:
print("==============================================================")
print("Summarize Total Score ")
dfFinalScore=dfSummary.rank(method='max',axis=0)
dfFinalScore['TotalScore']=dfFinalScore.sum(axis=1)
dfFinalScore.sort_values(by='TotalScore',ascending=False,inplace=True)
print(dfFinalScore)

Summarize Total Score 
        sum_return  sum_rank  TotalScore
symbol                                  
SPY            4.0       4.0         8.0
ACWI           3.0       3.0         6.0
GLD            2.0       2.0         4.0
AAXJ           1.0       1.0         2.0


In [24]:
if isPriceExcel:
    print("Export outcoome as Excel")
    with pd.ExcelWriter(f"{report_prefix_name}_{timeframe}_{from_str}To{to_str}.xlsx") as writer:
        dfPct.to_excel(writer, sheet_name=f"{timeframe}-Return")
        dfSum_Pct_Price.to_excel(writer, sheet_name="Return")
        dfRank.to_excel(writer, sheet_name="RANK")
        dfSummary.to_excel(writer, sheet_name="Summary")
        dfFinalScore.to_excel(writer, sheet_name="FinalScore")